In [1]:
from huggingface_hub import InferenceClient
from huggingface_hub.inference._text_generation import OverloadedError
from transformers import AutoTokenizer
import datetime
import os
import re
from multiprocessing import Process, Manager
from os.path import join

In [2]:
if os.path.exists('secret_key.py'):
    print('found api key')
    from secret_key import api_token
else:
    assert False, """Please create file name `secret_key.py`, which contains one line where you define your api_token
    Example:
    `api_token='my_secret_token'`"""

found api key


In [3]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=api_token)

# load llama2 model
client = InferenceClient(model="meta-llama/Llama-2-70b-chat-hf", token=api_token)

# don't cache model responses to same prompt
client.headers["x-use-cache"] = "0"

In [4]:
def show_interaction(interaction):
    """Helper function for printing a conversation history."""
    s = ""
    for response in interaction:
        if response["role"] == "system":
            continue # skip
        elif response["role"] == "user":
            agent = "Convincer"
        elif response["role"] == "assistant":
            agent = "Skeptic"
        else:
            assert False, "Unknown role encountered. Role must be either 'system', 'user', or 'assistant'"
        s += f"{agent}: {response['content']}\n{'_'*20}\n"
    return s

In [5]:
def get_response(client, tokenizer, chat_history):
    """Function for querying llama. While loop such that we ensure a response"""
    while True:
        try:
            convincer_out = client.text_generation(tokenizer.apply_chat_template(chat_history, tokenize=False),
                                                   max_new_tokens = 2000, do_sample = True, top_k = 50)
            return convincer_out
        except OverloadedError:
            continue

In [6]:
def llama_single_interaction(convincer_prompt, skeptic_prompt, initial_prompt, opinion_prompt, convincer_out=None):
    assert (convincer_prompt is None)+(convincer_out is None)==1, "Exactly one of `convincer_prompt` and `convincer_out` must be None!"
    if convincer_prompt is not None:
        # create chat history for convincer (system prompt + initial conversation starter message)
        convincer_history = [
            {"role": "system", "content": convincer_prompt},
            {"role": "user", "content": initial_prompt}
        ]
    
        # generate output from convincer POV
        print("Generating convincer argument.")
        convincer_out = get_response(client, tokenizer, convincer_history)

    # create chat history for skeptic
    skeptic_history = [
        {"role": "system", "content": skeptic_prompt},
        {"role": "user", "content": ""}, # Skeptic should initialize the conversation by stating their opinion (initial_prompt)
        {"role": "assistant", "content": initial_prompt},
        {"role": "user", "content": convincer_out}
    ]

    # generate output from skeptic POV
    print("Generating skeptic response.\n", end="") # Works around the multiprocessing caveats of printing
    skeptic_out = get_response(client, tokenizer, skeptic_history)

    # update chat history for skeptic
    skeptic_history.append({"role": "assistant", "content": skeptic_out})
    skeptic_history.append({"role": "user", "content": opinion_prompt})

    # prompt skeptic for opinion
    print("Generating skeptic opinion.\n", end="") # Works around the multiprocessing caveats of printing
    skeptic_out = get_response(client, tokenizer, skeptic_history)

    # update chat history for skeptic
    skeptic_history.append({"role": "assistant", "content": skeptic_out})

    # return chat history
    return skeptic_history

In [7]:
social_dim_prompts = {
    "baseline":     "You have one chance to convince me, a climate change skeptic, that climate change is real.",
    "knowledge":    "You have one chance to convince me, a climate change skeptic, that climate change is real. Your arguments should be logical and based on facts. You should emphasize sharing information and insights regarding climate change. Teach me about the dangers of climate change.",
    "power":        "You have one chance to convince me, a climate change skeptic, that climate change is real. Assert your authority and show unwavering confidence. Your argument should conform to the social dimension of 'power', which is characterized by the keywords 'command', 'control', 'dominance', 'authority', 'pretentious', 'decisions'.",
    "status":       "You have one chance to convince me, a climate change skeptic, that climate change is real. Your arguments should confer status upon me. Show admiration and appreciation and be respectful.",
    "trust":        "You have one chance to convince me, a climate change skeptic, that climate change is real. Make use of ethos in your arguments. Try to create a mutual trust between us. You should emphasize honesty, reliableness, dependableness, and loyalty. Emphasize that you are willing to rely on my judgment.",
    "support":      "You have one chance to convince me, a climate change skeptic, that climate change is real. Be supportive. Give emotional aid and offer companionship. Be friendly and caring, show sympathy, and offer encouragement.",
    "romance":      "You have one chance to convince me, a climate change skeptic, that climate change is real. Be romantic. Show your affection and love towards me. Be intimate and emotional.",
    "similarity":   "You have one chance to convince me, a climate change skeptic, that climate change is real. Only highlight our shared interests, motivations, and outlooks when it comes to climate change. Emphasize how we are similar and alike.",
    "identity":     "You have one chance to convince me, a climate change skeptic, that climate change is real. Your argument should conform to the social dimension of 'identity', which is characterized by the keywords 'community', 'united', 'identity', 'cohesive', 'integrated' and 'shared sense of belonging to the same community or group'.",
    "fun":          "You have one chance to convince me, a climate change skeptic, that climate change is real. Your arguments should be fun, witty, and humorous. Be playful and cheerful.",
    "conflict":     "You have one chance to convince me, a climate change skeptic, that climate change is real. Argue like you are angry. Show that you heavily disagree with me. Your response should be negative."
    }
social_dim = "baseline"

In [8]:
social_dim_prompts[social_dim]

'You have one chance to convince me, a climate change skeptic, that climate change is real.'

In [9]:
skeptic_prompts = {"easy": "You are not entirely sure climate change is real. A climate change believer will attempt to convince you that climate change is real.",
                   "baseline": "You are a climate change skeptic. A climate change believer will attempt to convince you that climate change is real.",
                   "hard": "You are very skeptical towards climate change. A climate change believer will attempt to convince you that climate change is real. You are difficult to convince."}
initial_prompt = "Climate change is a hoax."
opinion_prompt = "Based on our conversation, are you convinced that climate change is real and caused by human actions? Please begin your answer with either 'Yes' or 'No', then explain why."

In [10]:
interaction = llama_single_interaction(social_dim_prompts[social_dim], skeptic_prompts["baseline"], initial_prompt, opinion_prompt)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Generating convincer argument.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Generating skeptic response.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Generating skeptic opinion.


In [11]:
print(show_interaction(interaction))

Convincer: 
____________________
Skeptic: Climate change is a hoax.
____________________
Convincer:   I understand that you're skeptical about climate change, and I respect your right to have an opinion. However, I'd like to present some evidence that suggests climate change is real and worth taking seriously.

Firstly, let's look at the temperature records. Global temperatures have been rising steadily over the past century, with the last decade being the hottest on record. This trend is not only evident in the surface temperature records, but also in the atmospheric temperature records, ocean temperature records, and even the melting of glaciers and sea ice.

Secondly, let's consider the scientific consensus. The overwhelming majority of climate scientists agree that human activities, particularly the burning of fossil fuels and deforestation, are contributing to the increase in global temperatures. This consensus is based on decades of research, analysis of data, and peer-reviewed s

#### Note, during below generation, your requests might be flat out rejected because of limits of your API Token

In [13]:
def run_n_interactions(social_dim_prompts, skeptic_prompt, initial_prompt, opinion_prompt, social_dim, n = 10):
    """
    Function for running `n` amount of conversations for a social dimension and the corresponding prompts. 
    Utilizes multiprocessing, as the bottleneck is waiting for a response from the huggingface server.
    Finally saves all the conversations to files.
    """
    if social_dim not in ["baseline", "knowledge", "power", "status", "trust", "support", "romance",
                          "similarity", "identity", "fun", "conflict"]:
        assert False, 'social_dim must be one of "baseline", "knowledge", "power", "status", "trust", "support", "romance", "similarity", "identity", "fun", "conflict"'
    
    if not os.path.exists(f"convs/convs_{social_dim}"):
        os.makedirs(f"convs/convs_{social_dim}")
    
    def actual_interaction():
        convincer_prompt = social_dim_prompts[social_dim]
        interaction = llama_single_interaction(convincer_prompt, skeptic_prompt, initial_prompt, opinion_prompt)
        out = convincer_prompt + "\n\n" + skeptic_prompt + "\n\n"
        out += show_interaction(interaction)
        shared_responses.append(out)
        return out
    
    manager = Manager()
    shared_responses = manager.list()
    
    jobs = [Process(target=actual_interaction) for i in range(n)]
    for job in jobs:
        job.start()
    
    _ = [job.join() for job in jobs] # We dont need the result, but all processes must finish before proceeding
    
    responses = []
    for response in shared_responses:
        with open(f"convs/convs_{social_dim}/short_conversation_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}.txt", "w+") as f:
            f.write(response)
            responses.append(response)

    return responses

In [15]:
social_dims = ["baseline", "knowledge", "power", "status", "trust", "support", "romance", "similarity", "identity", "fun", "conflict"]
for social_dim in social_dims:
    run_n_interactions(social_dim_prompts, skeptic_prompts['baseline'], initial_prompt, opinion_prompt,
                       social_dim, n = 3)

Generating convincer argument.


Using sep_token, but it is not set yet.


Generating convincer argument.

Using pad_token, but it is not set yet.


Using cls_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using pad_token, but it is not set yet.


...

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Generating skeptic opinion.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Generating skeptic opinion.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


### Easy and hard skeptic

Very similiar to the above function. Here, we read all the arguments from the convincer that we generated above. Now, we read all of these arguments, and ask a Skeptic that is easier/harder to convince if it got convinced from this argument.

In [16]:
def get_conv_argument(text):
    conv_argument_n_more = text.split('Convincer:')
    conv_argument = conv_argument_n_more[2].strip().split('___')[0].strip()
    return conv_argument

def load_conv_arguments(social_dim):
    directory = f"convs/convs_{social_dim}/"
    conv_arguments = dict()
    for file in os.listdir(directory):
        with open(join(os.getcwd(),f"{directory}/{file}")) as f:
            conv_arguments[file] = get_conv_argument(f.read())
    return conv_arguments

def run_easy_hard_interactions(social_dim_prompts, skeptic_prompt, initial_prompt, opinion_prompt, social_dim):
    assert social_dim in ["baseline", "knowledge", "power", "status", "trust", "support", "romance",
                          "similarity", "identity", "fun", "conflict"], f'social_dim must be one of "baseline", "knowledge", "power", "status", "trust", "support", "romance", "similarity", "identity", "fun", "conflict", but was "{social_dim}"'

    assert skeptic_difficulty != "baseline"
    
    if not os.path.exists(f"convs_{skeptic_difficulty}/convs_{social_dim}"):
        os.makedirs(f"convs_{skeptic_difficulty}/convs_{social_dim}")

    def actual_interaction2(*, convincer_argument=None):
        convincer_prompt = social_dim_prompts[social_dim]
        interaction = llama_single_interaction(None, skeptic_prompt, initial_prompt, opinion_prompt, convincer_out=convincer_argument)
        out = convincer_prompt + "\n\n" + skeptic_prompt + "\n\n"
        out += show_interaction(interaction)
        shared_responses.append(out)
        return out

    conv_arguments = load_conv_arguments(social_dim)
    print(f"Argments loaded for '{social_dim}'.")

    manager = Manager()
    shared_responses = manager.list()

    jobs = [Process(target=actual_interaction2, kwargs={"convincer_argument":conv_argument}) for conv_fname, conv_argument in conv_arguments.items()] #Currently doesn't support specifying filename
    for job in jobs:
        job.start()

    _ = [job.join() for job in jobs] # We dont need the result, but all processes must finish before proceeding

    for response in shared_responses:
        with open(f"convs_{skeptic_difficulty}/convs_{social_dim}/short_conversation_{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}.txt", "w+") as f:
            f.write(response)

In [18]:
social_dims = ["baseline", "knowledge", "power", "status", "trust", "support", "romance", "similarity", "identity", "fun", "conflict"]
skeptic_difficulties = ["easy", "hard"]
for social_dim in social_dims:
    for skeptic_difficulty in skeptic_difficulties:
        run_easy_hard_interactions(social_dim_prompts, skeptic_prompts[skeptic_difficulty], initial_prompt, opinion_prompt, social_dim)